## 1️⃣ Verificar GPU e Configurar Ambiente

In [ ]:
# Verificar se GPU está disponível
import torch
print(f"🔍 GPU Disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️ GPU não detectada! Verifique Runtime > Change runtime type > GPU")

## 2️⃣ Montar Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive montado!")

## 3️⃣ Clonar Repositório do GitHub

In [ ]:
# Clonar repositório
!git clone https://github.com/mlisboa17/projetos-logos.git
%cd projetos-logos
!pwd
print("\n✅ Repositório clonado!")

## 4️⃣ Instalar Dependências

In [ ]:
# Instalar ultralytics (YOLO) e dependências
!pip install ultralytics -q
!pip install pillow opencv-python-headless numpy -q

# Verificar instalação
from ultralytics import YOLO
print("✅ Dependências instaladas!")

## 5️⃣ Baixar Dataset do Google Drive

**IMPORTANTE:** Você precisa ter feito upload do dataset para o Google Drive primeiro!

### Opção A: Se o dataset está no Drive como `dataset_treino.zip`

In [ ]:
# EDITE O CAMINHO para onde você salvou o dataset no Google Drive
DATASET_PATH = "https://drive.google.com/drive/my-drive/dataset_treino.zip"  # ⚠️ AJUSTE ESTE CAMINHO!

# Copiar e descompactar
!cp "$DATASET_PATH" /content/dataset_treino.zip
!unzip -q /content/dataset_treino.zip -d /content/projetos-logos/verifik/
!ls -lh /content/projetos-logos/verifik/dataset_treino/
print("\n✅ Dataset descompactado!")

### Opção B: Download direto via gdown (se compartilhou link público)

In [ ]:
# Se você tiver link público do Google Drive
# !pip install gdown -q
# !gdown --id SEU_ID_DO_GOOGLE_DRIVE -O dataset_treino.zip
# !unzip -q dataset_treino.zip -d /content/projetos-logos/verifik/
# print("✅ Dataset baixado!")

## 6️⃣ Preparar Configuração do Treinamento

In [ ]:
import os
import yaml

# Verificar estrutura do dataset
dataset_path = '/content/projetos-logos/verifik/dataset_treino/20251124_211122'

if os.path.exists(dataset_path):
    print(f"✅ Dataset encontrado: {dataset_path}")
    print(f"\n📁 Estrutura:")
    !ls -lh {dataset_path}/images/train/ | head -10
    !ls -lh {dataset_path}/labels/train/ | head -10
else:
    print("❌ Dataset não encontrado! Verifique o caminho.")
    !ls -la /content/projetos-logos/verifik/

In [ ]:
# Criar arquivo de configuração YAML
config = {
    'path': '/content/projetos-logos/verifik/dataset_treino/20251124_211122',
    'train': 'images/train',
    'val': 'images/train',  # Usando mesmo dataset para validação (ajuste se tiver separado)
    'names': {
        0: 'HEINEKEN',
        1: 'BUDWEISER',
        2: 'AMSTEL',
        3: 'STELLA',
        4: 'DEVASSA',
        5: 'PETRA',
        6: 'PILSEN',
        7: 'PEPSI',
        8: 'COCA',
        9: 'REFRIGERANTE'
    }
}

# Salvar configuração
config_file = '/content/dataset_config.yaml'
with open(config_file, 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print("✅ Configuração criada!")
print(f"\n📄 Conteúdo de {config_file}:")
!cat {config_file}

## 7️⃣ Treinar Modelo YOLO

**Parâmetros:**
- `model='yolov8n.pt'` - Modelo nano (rápido, 6MB)
- `epochs=50` - Número de épocas (ajuste conforme necessário)
- `imgsz=640` - Tamanho da imagem
- `batch=16` - Batch size (reduza se der erro de memória)
- `device=0` - Usa GPU (0 = primeira GPU)

In [ ]:
from ultralytics import YOLO
import time

# Carregar modelo base
model = YOLO('yolov8n.pt')  # Modelo nano
# model = YOLO('yolov8s.pt')  # OU modelo small (mais preciso, mais lento)

print("🚀 Iniciando treinamento...\n")
start_time = time.time()

# Treinar
results = model.train(
    data=config_file,
    epochs=50,           # ⚠️ AJUSTE: Mais épocas = melhor modelo (mas mais tempo)
    imgsz=640,
    batch=16,            # ⚠️ Se der erro de memória, reduza para 8 ou 4
    device=0,            # GPU
    patience=10,         # Early stopping
    save=True,
    project='runs/treino_colab',
    name='experimento_1',
    exist_ok=True,
    verbose=True
)

elapsed_time = time.time() - start_time
print(f"\n✅ Treinamento concluído em {elapsed_time/60:.1f} minutos!")

## 8️⃣ Visualizar Resultados

In [ ]:
from IPython.display import Image, display

# Caminho dos resultados
results_path = 'runs/treino_colab/experimento_1'

print("📊 RESULTADOS DO TREINAMENTO\n")
print("="*50)

# Exibir gráficos
print("\n📈 Curvas de Loss e Métricas:")
display(Image(filename=f'{results_path}/results.png', width=800))

print("\n🎯 Matriz de Confusão:")
display(Image(filename=f'{results_path}/confusion_matrix.png', width=600))

print("\n📊 Curvas de Precisão-Recall:")
display(Image(filename=f'{results_path}/PR_curve.png', width=600))

print("\n🔍 Exemplos de Validação:")
display(Image(filename=f'{results_path}/val_batch0_pred.jpg', width=800))

## 9️⃣ Testar Modelo em Imagem de Exemplo

In [ ]:
# Carregar modelo treinado
best_model = YOLO('runs/treino_colab/experimento_1/weights/best.pt')

# Testar em imagem do dataset
test_image = '/content/projetos-logos/verifik/dataset_treino/20251124_211122/images/train/HEINEKEN_CERVEJA_HEINEKEN_330ML_0.jpg'

# Fazer predição
results = best_model.predict(test_image, conf=0.25, save=True)

print("\n✅ Teste concluído!")
print(f"📁 Resultado salvo em: runs/detect/predict/")

# Exibir resultado
from IPython.display import Image, display
display(Image(filename='runs/detect/predict/HEINEKEN_CERVEJA_HEINEKEN_330ML_0.jpg', width=600))

## 🔟 Salvar Modelo no Google Drive

In [ ]:
import shutil
from datetime import datetime

# Criar nome com timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_name = f"best_model_{timestamp}.pt"

# Copiar para Google Drive
source = 'runs/treino_colab/experimento_1/weights/best.pt'
destination = f'/content/drive/MyDrive/{model_name}'

shutil.copy(source, destination)

print(f"✅ Modelo salvo no Google Drive!")
print(f"📁 Localização: {destination}")
print(f"📊 Tamanho: {os.path.getsize(source) / (1024**2):.1f} MB")
print(f"\n💡 Baixe este arquivo e coloque em:")
print(f"   verifik/runs/treino_continuado/weights/best.pt")

## 📊 Informações do Modelo

In [ ]:
# Exibir informações do modelo
print("📊 INFORMAÇÕES DO MODELO TREINADO\n")
print("="*50)

# Arquivos gerados
print("\n📁 Arquivos gerados:")
!ls -lh runs/treino_colab/experimento_1/weights/

# Métricas finais (se disponível)
try:
    with open('runs/treino_colab/experimento_1/results.csv', 'r') as f:
        lines = f.readlines()
        print("\n📈 Últimas métricas:")
        print(lines[0].strip())  # Header
        print(lines[-1].strip())  # Última época
except:
    print("\n⚠️ Métricas não disponíveis")

print("\n✅ Treinamento completo!")

## 📝 Próximos Passos

1. **Baixar modelo treinado** do Google Drive
2. **Testar localmente** com `detector_simples.py`
3. **Avaliar performance** em imagens reais
4. **Retreinar** se necessário com mais dados

---

## 🆘 Troubleshooting

### GPU não detectada
```
Runtime → Change runtime type → GPU → T4 GPU → Save
```

### Erro de memória (CUDA out of memory)
```python
# Reduza o batch_size:
batch=8  # ou batch=4
```

### Sessão expirou (12h)
```
- Modelo salvo no Drive está seguro!
- Re-execute do início se precisar continuar
```

---

**🎉 Bom treinamento!**